In [1]:
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [14]:
path='dogs-vs-cats/train'

In [15]:
## used for resize and in our model
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [16]:
## Definition to load all our dog images
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [17]:
## Definition to load all our cat images
def load_cats():
    print('Loading all cat images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [18]:
load_dogs()

Loading all dog images

All dog images loaded


In [19]:
load_cats()

Loading all cat images

All cat images loaded


In [20]:
classes = ['dog', 'cat']

In [21]:
## just change the labels for 0 and  1
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [22]:
## converting everything to Numpy array to fit in our model
## them creating a X and target file like we used to see
## in Machine and Deep Learning models
dogs = np.asarray(dogs).astype('float32')
cats = np.asarray(cats).astype('float32')
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [23]:
## fit values between 0 and 1
dogs /= 255
cats /= 255

In [25]:
x = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [26]:
from sklearn.model_selection import train_test_split
# train set 10000개, test_set 25000개
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=0)

In [28]:
x_train.shape

(20000, 96, 96, 1)

In [29]:
x_test.shape

(5000, 96, 96, 1)

In [32]:
## One-Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [33]:
y_train.shape

(20000, 2)

In [34]:
y_test.shape

(5000, 2)

In [35]:
## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, IMG_CHANNEL), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.5),
    Dense(N_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)       

In [36]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(), metrics=['accuracy'])  # 'adam'을 import 해서 클레스로 씀

In [38]:
## to save checkpoint to use later
modelpath = "model/dogs_vs_cats-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=2, save_best_only=True)

In [42]:
model.fit(x_train, y_train, batch_size=50, epochs=10, validation_split=0.2,
          callbacks=[checkpointer])

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
15950/16000 [============================>.] - ETA: 0s - loss: 0.2748 - accuracy: 0.8813
Epoch 00001: val_loss did not improve from 0.45000
16000/16000 [==============================] - 233s 15ms/sample - loss: 0.2752 - accuracy: 0.8811 - val_loss: 0.4782 - val_accuracy: 0.8015
Epoch 2/10
15950/16000 [============================>.] - ETA: 0s - loss: 0.2224 - accuracy: 0.9072
Epoch 00002: val_loss did not improve from 0.45000
16000/16000 [==============================] - 233s 15ms/sample - loss: 0.2225 - accuracy: 0.9071 - val_loss: 0.5399 - val_accuracy: 0.8045
Epoch 3/10
15950/16000 [============================>.] - ETA: 0s - loss: 0.1725 - accuracy: 0.9315
Epoch 00003: val_loss did not improve from 0.45000
16000/16000 [==============================] - 234s 15ms/sample - loss: 0.1726 - accuracy: 0.9316 - val_loss: 0.5337 - val_accuracy: 0.8000
Epoch 4/10
15950/16000 [============================>.] - ETA: 0s - loss: 0.14

KeyError: 'val_loss'

In [ ]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/dogs_vs_cats-cnn-06-0.4500.hdf5')

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])